In [3]:
using ModelingToolkit, Plots, DifferentialEquations, Revise, ModelingToolkitStandardLibrary, Unitful
@variables t [unit = u"s"]

1-element Vector{Num}:
 t

In [39]:

showeq(x) = equations(expand_connections(x))

@connector function ThermoPin(; name)
    @variables P(t)=1.0 [unit = u"bar"] 
    @variables ṁ(t)=1.0 [connect = Flow, unit = u"kg/s"] 
    @variables T(t)=293.15 [unit = u"K"]

    sts = [T, P, ṁ]
    ODESystem(Equation[], t, sts, []; name = name)
end

@connector function ThermoGround(; name)
    @named gnd = ThermoPin()
    @parameters pzero = 0 [unit = u"bar"] Tzero = 273.15 [unit = u"K"]
    eqs = [gnd.P ~ pzero]
    compose(ODESystem(eqs, t, [], []; name = name), gnd)
end

@component function ThermoOnePort(; name)
    @named p = ThermoPin()
    @named n = ThermoPin()
    sts = @variables P(t)=0.0 [unit = u"bar"] T(t)=0.0 [unit = u"K"] ṁ(t) = 1.0 [unit = u"kg/s"]
    eqs = [P ~ p.P - n.P
            T ~ p.T - n.T
            0 ~ p.ṁ + n.ṁ
            ṁ ~ p.ṁ]
    compose(ODESystem(eqs, t, sts, []; name = name), p, n)
end


ThermoOnePort (generic function with 1 method)

In [40]:
@component function FlowSource(;name, mflow = 1.0u"kg/s")
    @named prt = ThermoOnePort()
    ps = @parameters mflow=mflow [unit = u"kg/s"]
    eqs = [prt.ṁ ~ mflow]
    extend(ODESystem(eqs, t, [], ps; name = name),prt)
end
@component function PressureSource(;name, press = 100u"bar")
    @named prt = ThermoOnePort()
    ps = @parameters P=press [unit = u"bar"]
    eqs = [prt.P ~ P]
    extend(ODESystem(eqs, t, [], ps; name = name),prt)
end
@named mtest = FlowSource()
@named g = ThermoGround()
con = connect(g.gnd,mtest.n)
@named sys = ODESystem(con,t; systems = [mtest,g])
showeq(sys)
structural_simplify(sys)
showeq(sys)

┌ Warning: p contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: n contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40


┌ Warning: gnd contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40


12-element Vector{Equation}:
 mtest₊P(t) ~ mtest₊p₊P(t) - mtest₊n₊P(t)
 mtest₊T(t) ~ mtest₊p₊T(t) - mtest₊n₊T(t)
 0 ~ mtest₊n₊ṁ(t) + mtest₊p₊ṁ(t)
 mtest₊ṁ(t) ~ mtest₊p₊ṁ(t)
 mtest₊prt₊ṁ(t) ~ mtest₊mflow
 g₊gnd₊P(t) ~ g₊pzero
 0 ~ g₊gnd₊ṁ(t)
 g₊gnd₊T(t) ~ mtest₊n₊T(t)
 g₊gnd₊P(t) ~ mtest₊n₊P(t)
 0 ~ mtest₊n₊ṁ(t) - g₊gnd₊ṁ(t)
 0 ~ mtest₊p₊ṁ(t)
 0 ~ g₊gnd₊ṁ(t)

In [43]:
@component function FlowResistance(;name, R = 1.0u"1/m/s")
    @named op = ThermoOnePort()
    @unpack P,T, ṁ = op
    ps = @parameters R = R [unit = u"bar/(kg/s)"]
    eqs = [
        P ~ ṁ * R,
        0 ~ T
    ]
    extend(ODESystem(eqs, t, [], ps; name = name), op)
end

@named mtest = PressureSource()
@named g = ThermoGround()
@named fr = FlowResistance()

con = [connect(g.gnd,mtest.n)
        connect(mtest.p,fr.p)]

@named sys = ODESystem(con,t; systems = [mtest,g,fr])
showeq(sys)
structural_simplify(sys)
showeq(sys)

┌ Warning: p contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: n contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: gnd contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia

┌ Warning: n contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40


21-element Vector{Equation}:
 mtest₊P(t) ~ mtest₊p₊P(t) - mtest₊n₊P(t)
 mtest₊T(t) ~ mtest₊p₊T(t) - mtest₊n₊T(t)
 0 ~ mtest₊n₊ṁ(t) + mtest₊p₊ṁ(t)
 mtest₊ṁ(t) ~ mtest₊p₊ṁ(t)
 mtest₊prt₊P(t) ~ mtest₊P
 g₊gnd₊P(t) ~ g₊pzero
 fr₊P(t) ~ fr₊p₊P(t) - fr₊n₊P(t)
 fr₊T(t) ~ fr₊p₊T(t) - fr₊n₊T(t)
 0 ~ fr₊n₊ṁ(t) + fr₊p₊ṁ(t)
 fr₊ṁ(t) ~ fr₊p₊ṁ(t)
 ⋮
 0 ~ g₊gnd₊ṁ(t)
 g₊gnd₊T(t) ~ mtest₊n₊T(t)
 g₊gnd₊P(t) ~ mtest₊n₊P(t)
 0 ~ mtest₊n₊ṁ(t) - g₊gnd₊ṁ(t)
 mtest₊p₊T(t) ~ fr₊p₊T(t)
 mtest₊p₊P(t) ~ fr₊p₊P(t)
 0 ~ fr₊p₊ṁ(t) + mtest₊p₊ṁ(t)
 0 ~ g₊gnd₊ṁ(t)
 0 ~ fr₊n₊ṁ(t)

In [41]:
mtk = ModelingToolkit
@component function ThermoCapacitance(;name, cp=5.192u"kJ/(kg/s)", m =100u"kg")
    # C = mcp = ρVcp
    @named op = ThermoOnePort()
    @unpack T,P, ṁ = op
    D = Differential(t)
    ps = @parameters cp = cp [unit = u"kJ/kg/K"] m = m [unit = u"kg"] Cf = 100 [unit = u"kg/bar"]
    display(mtk.get_unit(ṁ*cp*T))
    display(mtk.get_unit(D(P)))
    display(mtk.get_unit(ṁ/Cf))
    eqs = [
        D(T) ~ ṁ/m*T
        D(P) ~ ṁ/Cf
    ]
    extend(ODESystem(eqs, t, [], ps; name = name), op)
end

@named psrc= PressureSource()
@named g = ThermoGround()
@named fr = FlowResistance()
@named tc = ThermoCapacitance()

con = [ connect(psrc.p,fr.p)
        connect(fr.n,tc.p)
        connect(tc.n,psrc.n)
        connect(tc.n,g.gnd)]

@named _model = ODESystem(con, t)
@named syse = compose(_model,[mtest,g,fr,tc])
showeq(syse)
sys2 = structural_simplify(syse)


┌ Warning: p contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: n contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: gnd contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia

UndefVarError: UndefVarError: FlowResistance not defined

In [38]:
using NonlinearSolve
@variables temps
@register_symbolic cpfunc(temps)
@register_symbolic kfunc(temps)

@connector function FlowPin(; name)
    sts = @variables P=1.0 ṁ=1.0 [connect = Flow]
    ps = @parameters cp k
    NonlinearSystem(Equation[],  sts, []; name = name)
end

@connector function HeatPin(; name)
    sts = @variables T=300 Q̇=0.0 [connect = Flow]
    NonlinearSystem(Equation[],  sts, []; name = name)
end
@connector function WorkPin(; name)
    sts = @variables Ẇ = 0.0
    NonlinearSystem(Equation[], sts, []; name = name)
end

# convention = +Q = Qin
@connector function HeatFlowPin(; name, Qin = 0.0)
    sts = @variables Q̇=Qin [input = true]
    ps = @parameters Qin = Qin
    eqs=[
        Q̇ ~ Qin
    ]
    NonlinearSystem(eqs, sts, ps; name = name)
end

# convention = +W = Win
@connector function WorkPort(; name)
    sts = @variables Ẇ = 0.0 [output = true]
    NonlinearSystem(Equation[],  sts, []; name = name)
end

@connector function ThermoPin(; name)
    sts = @variables P=1.0 ṁ=1.0 T=293.15 
    ps = @parameters cp = 5192 k=1.667
    NonlinearSystem(Equation[],  sts, ps; name = name)
end

@component function FlowSource(;name, mflow = 1.0, P = 10, T = 300)
    @named src = ThermoPin()
    ps = @parameters mflow=mflow P=P T=T
    eqs = [
        src.T ~ T
        src.P ~ P
        src.ṁ ~ mflow
    ]
    compose(NonlinearSystem(eqs,  [], ps; name = name),src)
end

@component function ThermoGround(; name)
    @named gnd = ThermoPin()
    eqs = [gnd.P ~ 0;   gnd.T ~ 273.15]
    compose(NonlinearSystem(eqs,  [], []; name = name), gnd)
end

@component function ThermoHeat(; name)
    @named p = ThermoPin()
    @named n = ThermoPin()
    @named h = HeatFlowPin()

    eqs = [
        p.ṁ ~ n.ṁ                               # conservation of mass
        n.T ~ p.T + h.Q̇/(p.ṁ*p.cp)
        n.P ~ p.P
    ]
    compose(NonlinearSystem(eqs,[],[]; name = name), p,n,h)
end

@component function ThermoCompressor(; name, η = 1.0, rp = 3.5)
    @named p = ThermoPin()
    @named n = ThermoPin()
    @named w = WorkPin()

    ps = @parameters rp = rp η = η
    eqs = [
        p.ṁ ~ n.ṁ                               # conservation of mass
        n.P ~ p.P * rp
        n.T ~ p.T * (η + rp^((p.k-1)/p.k)- η*(rp^((p.k-1)/p.k))) / (rp^((p.k-1)/p.k))
        w.Ẇ ~ p.ṁ * p.cp * (n.T - p.T)
    ]
    compose(NonlinearSystem(eqs,[],ps; name = name), p,n,w)
end

@component function ThermoTurbine(; name, η = 1.0, rp = 3.5)
    @named p = ThermoPin()
    @named n = ThermoPin()
    @named w = WorkPin()

    ps = @parameters rp = rp η = η
    eqs = [
        p.ṁ ~ n.ṁ                               # conservation of mass
        n.P ~ p.P * rp
        n.T ~ p.T * ((1.0 - η - (rp^((p.k-1)/p.k))) / (-η))
        w.Ẇ ~ p.ṁ * p.cp * (n.T - p.T)
    ]
    compose(NonlinearSystem(eqs,[],ps; name = name), p,n,w)
end

@named mf = FlowSource();
@named cc = ThermoCompressor()
@named qq = ThermoHeat()
@named tt = ThermoTurbine()

con = [connect(mf.src,cc.p)
        connect(cc.n,qq.p)
        connect(qq.n,tt.p)]

        
@named connected_sys =  NonlinearSystem(con,[],[]; systems = [mf, cc, qq, tt]);
# updaded_eq = substitute(equations(expand_connections(connected_sys)))
@named newsys = NonlinearSystem(equations(expand_connections(connected_sys)),states(connected_sys),parameters(connected_sys))

p = [mf.mflow => 100
    mf.P => 10
    mf.T => 300
    cc.rp => 3.5
    cc.η => .9
    qq.h.Qin => 10e6
    tt.rp => 3.5
    tt.η => 0.9]

showeq(connected_sys)



# Symbolics.build_function(equations(newsys),parameters(newsys))

# simple_sys = structural_simplify(newsys)
showeq(newsys)
simplify(newsys)
# full_equations(simple_sys)
prob = NonlinearProblem(newsys,[],p)
# solve(simple_sys)
NonlinearSolve(prob)
# solve(prob)


┌ Warning: src contains 0 flow variables, yet 3 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: p contains 0 flow variables, yet 3 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: n contains 0 flow variables, yet 3 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia

┌ Warning: p contains 0 flow variables, yet 3 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: n contains 0 flow variables, yet 3 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40


┌ Warning: p contains 0 flow variables, yet 3 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: n contains 0 flow variables, yet 3 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\packages\ModelingToolkit\FbXPg\src\systems\connectors.jl:40
┌ Warning: w contains 0 flow variables, yet 1 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\harvey\.julia\p

MethodError: MethodError: objects of type Module are not callable

In [279]:

@connector function ThermoPins(; name)
    sts = @variables P=1.0 ṁ=1.0 T=293.15 cp(T)=0.0 k(T)=0.0
    ps = @parameters cp(T) k(T)
    NonlinearSystem(Equation[],  sts, ps; name = name)
end



# fprops(x)=fprops(cp -> cpf(cp), k -> kk(k))
f = fprops(cpf,kk)
f.cp(10)
@unpack cp = f

@register_symbolic cppp(T)
@register_symbolic kkkk(T)


In [ ]:
@component function MassFlowSource(; name,  Ṁ = 1.0 , Pin=10 , Tin = 300)
    @named tp = ThermoPort()
    ps = @parameters Ṁ = Ṁ Pin = Pin Tin = Tin
    eqs = [
            0 ~ tp.ṁ - Ṁ
            ]
    extend(NonlinearSystem(eqs,  [], ps; name = name),tp)
end



@component function ThermoPort(; name)
    @named inlet = ThermoPin()
    @named outlet = ThermoPin()
    @named q = HeatFlowPin()
    @named w = WorkPin()

    sts = @variables P=0.0 T=0.0 ṁ=1.0

    eqs = [ T ~ outlet.T - inlet.T
            P ~ outlet.P - inlet.P
            inlet.ṁ ~ outlet.ṁ
            ṁ ~ inlet.ṁ]

    compose(NonlinearSystem(eqs,  sts, []; name = name),inlet,outlet,q,w)
end

@component function PassiveThermoPort(; name)
    @named inlet = ThermoPin()
    @named outlet = ThermoGround()

    sts = @variables P=0.0 T=0.0 ṁ=1.0

    eqs = [
        0 ~ inlet.T - T
        P ~ inlet.P - P
        0 ~ inlet.ṁ - outlet.gnd.ṁ
        0 ~ inlet.ṁ - ṁ
    ]
    # extend(ODESystem(eqs, t, [], []; name = name), tp)
    compose(NonlinearSystem(eqs, sts, []; name = name),inlet,outlet)
end

# compression ratio is always A --> B
@component function gas_compressor(;name)
    @named tp = ThermoPort()
    ps = @parameters rp η cp k
    kcoeff = (k-1)/k
    eqs = [
        0 ~ rp* tp.inlet.P - tp.P
        0 ~ tp.inlet.T*(1 - rp^kcoeff)/(-η) - tp.T
        0 ~ tp.w.Ẇ - tp.ṁ * cp * tp.T
        0 ~ tp.q.Q̇
    ]
    # extend(ODESystem(eqs, t,[], ps; name = name), tp)
    extend(NonlinearSystem(eqs,  [], ps; name = name),tp)
end

# compression ratio is always A --> B
@component function gas_turbine(;name)
    @named tp = ThermoPort()
    ps = @parameters rp η cp k
    kcoeff = (k-1)/k
    eqs = [
        0 ~ rp* tp.inlet.P - tp.P
        0 ~ η*(rp^kcoeff-1)*tp.inlet.T - tp.T 
        0 ~ tp.w.Ẇ - tp.ṁ * cp * T
        0 ~ tp.q.Q̇
    ]
    # extend(ODESystem(eqs, t,[], ps; name = name), tp)
    extend(NonlinearSystem(eqs,  [], ps; name = name),tp)
end

@component function MassFlowSource(; name,  Ṁ = 1.0 , Pin=10 , Tin = 300 )
    @named tp = ThermoPort()
    ps = @parameters Ṁ = Ṁ Pin = Pin Tin = Tin
    eqs = [
            0 ~ tp.ṁ - Ṁ
            ]
    extend(NonlinearSystem(eqs,  [], ps; name = name),tp)
end


In [ ]:
@named g1 = ThermoGround();
@named mf = MassFlowSource();
@named cs = gas_compressor();
# @named ptm = ThermoPort();
# @named g2 = ThermoGround()
# con = [connect(g1.gnd,mf.inlet)]




con = [connect(g1.gnd,mf.inlet)
        connect(mf.outlet,cs.inlet)]
    # connect(cs.outlet,ptm.inlet)
    # connect(ptm.outlet,g2.gnd)]
# connected_sys =  NonlinearSystem(con,[],[]; systems = [g1,mf])


@named connected_sys =  NonlinearSystem(con,[],[]);
@named comp_sys = compose(g1,mf,cs,ptm,g2)
@named full_sys = compose(comp_sys,connected_sys)

# for sys in [mf,cs,ptm]
#     println(sys.name)
#     connected_sys =  NonlinearSystem(con,[],[]; systems = [g1,mf,cs])
#     structural_simplify(sys)
# end
display(equations(expand_connections(connected_sys)))
display(equations(expand_connections(comp_sys)))
# states(connected_sys)

simple_sys = structural_simplify(full_sys);
# display(equations(simple_sys))
# display(equations(expand_connections(connected_sys)))
# states(connected_sys)
# equations(g1)
# get_variables(simple_sys)

In [ ]:


@named odesys = NonlinearSystem(con,[],[]; systems = [g1,mf,cs,ptm])

    @named nlsys = NonlinearSystem(equations(expand_connections(odesys)), states(odesys),parameters(odesys))

